In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

import math

In [2]:
def read(path):
  return mpimg.imread(path)

def show(image):
  print('This image is:', type(image), 'with dimensions:', image.shape)
  plt.imshow(image)

def save(image, path = 'test.png'):
  plt.imsave(path, image)

def convert_to_color(image):
  return np.dstack((image, image, image))

In [3]:
def gray(raw_image):
  '''convert the colored imaged to grayscale image'''
  return cv2.cvtColor(raw_image, cv2.COLOR_RGB2GRAY)  

def reduce_noise(gray_image, kernel_size):
  '''reduce noise of grayscale image using gausian blur'''
  return cv2.GaussianBlur(gray_image, (kernel_size, kernel_size), 0)

In [4]:
def get_edges(blur_image, low_threshold, high_threshold):
  '''find edges using canny transform algorithm'''
  return cv2.Canny(blur_image, low_threshold, high_threshold)

In [5]:
def get_vertices(x, y, axc, bxc, cyc, dyc, maxyc = 1.0, maxxc = 1.0, startxc = 0.0):
  ax = int(axc*x)
  bx = int(bxc*x)
  cy = int(cyc*y)
  dy = int(dyc*y)
  maxy = int(maxyc*y)
  maxx = int(maxxc*x)
  startx = int(startxc*x)
  bottom_left = (startx, maxy)
  top_left = (ax, cy)
  top_right = (bx, dy)
  bottom_right = (maxx, maxy)
  vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
  return vertices 

In [6]:
def get_roi(edge_image, vertices, ignore_value = 255):
  '''get the polygon to be used to block out everything in the image except the region of interest'''
  roi = np.zeros_like(edge_image)
  cv2.fillPoly(roi, vertices, ignore_value)
  return roi

def mask(edge_image, roi):
  '''block out everything in the image except the edges in the region of interest'''
  return cv2.bitwise_and(edge_image, roi)

In [7]:
def get_lines(masked_edge_image, rho, theta_coef, min_votes, min_line_length, max_line_gap):
  '''convert edges into lines using hough transform algorithm '''
  theta = theta_coef*np.pi/180
  return cv2.HoughLinesP(masked_edge_image, rho, theta, min_votes, np.array([]), minLineLength = min_line_length, maxLineGap = max_line_gap)


In [8]:
def draw(lines, image, color=[255, 0, 0], thickness = 2, thresh = 0.5):
  ''' draw the lines on a blank image'''
  lined_image = np.copy(image)*0
  
  if lines is not None:
    for line in lines:
      for x1,y1,x2,y2 in line:
        slope, intercept = np.polyfit((x1,x2), (y1,y2), 1)
        if abs(slope) > thresh:
          cv2.line(lined_image, (x1, y1), (x2, y2), color, thickness)
        
  return lined_image

In [9]:
# equation of a line: y = slope*x + intercept 
# left lane has a positive slope, right lane has a negative slope 

def extrapolate_lines(lines, image, color=[255, 0, 0], thickness = 10, 
                      positive_thresh = 0.5, negative_thresh = 0.5):
    
  imshape = image.shape
  image = np.copy(image)*0
    
  #initialize minimum and maximum y coordinate 
  minimum_y = image.shape[0] 
  maximum_y = image.shape[0]
  
  #initialize groups of values into empty lists
  left_slopes = []
  left_xs = []
  left_ys = []   
  right_slopes = []
  right_xs = []
  right_ys = []
  
  # segregate the small line segments into the left lane group or right lane group
  if lines is not None:  
    for line in lines:
      for x1,y1,x2,y2 in line:
        
        # get the slope and intercept of the line (as defined by two points) using the polyfit function
        slope, intercept = np.polyfit((x1,x2), (y1,y2), 1)
            
        if (slope > positive_thresh): #if positive slope, put value to left lane group
          left_slopes += [slope]
          left_xs += [x1, x2]
          left_ys += [y1, y2]
        elif (slope < negative_thresh): #if negative slope, put value to right lane group
          right_slopes += [slope]
          right_xs += [x1, x2]
          right_ys += [y1, y2]
        
        # update the minimum y_coordinate based on values seen
        minimum_y = min(min(y1, y2), minimum_y)
  
  #average all the values in each group to get the slope, x, and y
  left_slope = np.mean(left_slopes)
  left_x = np.mean(left_xs)
  left_y = np.mean(left_ys)
  right_slope = np.mean(right_slopes)
  right_x = np.mean(right_xs)
  right_y = np.mean(right_ys)
    
  #derive the intercept using the equation of the line and average value
  left_intercept = left_y - (left_slope * left_x)
  right_intercept = right_y - (right_slope * right_x)

  if ((len(left_slopes) > 0) and (len(right_slopes) > 0)): #make sure we have points in each group
    #derive the x coordinate using the equation of the lines and derived values
    upper_left_x = int((minimum_y - left_intercept) / left_slope)
    lower_left_x = int((maximum_y - left_intercept) / left_slope)
    upper_right_x = int((minimum_y - right_intercept) / right_slope)
    lower_right_x = int((maximum_y - right_intercept) / right_slope)
    
    #draw the line based on two points 
    cv2.line(image, (upper_left_x, minimum_y), (lower_left_x, maximum_y), color, thickness)
    cv2.line(image, (upper_right_x, minimum_y), (lower_right_x, maximum_y), color, thickness)
    
  return image

In [10]:
def overlap(first_image, second_image, α = 0.8, β = 0.5, λ = 0.0):
    '''first_image * α + second_image * β + λ (colored)'''
    return cv2.addWeighted(first_image, α, second_image, β, λ)

In [11]:
param = {
   #blur parameters
  'kernel_size': 5, 
    
   #canny transform parameters
  'canny_lo': 100, 
  'canny_hi': 200, 

  #region of interest parameters
  'ax_coef': 10.0/25,
  'bx_coef': 14.0/25,
  'cy_coef': 0.6,
  'dy_coef': 0.6,
  'maxy_coef': 1.0,
  'maxx_coef': 1.0,
  'startx_coef': 0.0,

  #hough parameters
  'rho': 1, 
  'theta_coef': 1, 
  'min_votes': 30, 
  'min_line_length': 20, 
  'max_line_gap': 20
}

In [12]:
def pipeline(path, file, name, param = param):
    
  name = path + name

  raw_image = read(path + file)
  image = read(path + file)
  save(image, path = name + '1-raw.png')
  
  gray_image = gray(image)
  save(gray_image, path = name + '2-grey.png')
    
  blur_image = reduce_noise(gray_image, param['kernel_size'])
  save(blur_image, path = name + '3-blur.png')
    
  edge_image = get_edges(blur_image, param['canny_lo'], param['canny_hi'])
  save(edge_image, path = name + '4-edges.png')
  
  x = image.shape[1]
  y = image.shape[0]
  vertices = get_vertices(x, y, param['ax_coef'], param['bx_coef'],
                                param['cy_coef'], param['dy_coef'],
                                param['maxy_coef'], param['maxx_coef'], param['startx_coef'])
  
  roi = get_roi(edge_image, vertices)
  save(roi, path = name + '5-roi.png')
  
  mask_image = mask(edge_image, roi)
  save(mask_image, path = name + '6-mask.png')
  
  lines = get_lines(mask_image, param['rho'], param['theta_coef'],
                           param['min_votes'], param['min_line_length'],
                           param['max_line_gap'])
   
  line_image = draw(lines, raw_image)
  save(line_image, path = name + '7-lines.png')
  
  
  line_image2 = extrapolate_lines(lines, raw_image)
  save(line_image2, path = name + '8-lines.png')

  image = overlap(line_image, raw_image)
  save(image, path = name + '9-overlap.png')

  image = overlap(line_image2, raw_image)
  save(image, path = name + '10-overlap.png')
   

pipeline(path = 'test_images/', file = 'solidWhiteCurve.jpg', name = 'A')
pipeline(path = 'test_images/', file = 'solidWhiteRight.jpg', name = 'B')
pipeline(path = 'test_images/', file = 'solidYellowCurve.jpg', name = 'C')
pipeline(path = 'test_images/', file = 'solidYellowCurve2.jpg', name = 'D')
pipeline(path = 'test_images/', file = 'solidYellowLeft.jpg', name = 'E')
pipeline(path = 'test_images/', file = 'whiteCarLaneSwitch.jpg', name = 'F')



In [13]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [14]:
def process_image(image):

  raw_image = np.copy(image)
  gray_image = gray(image)    
  blur_image = reduce_noise(gray_image, param['kernel_size'])
    
  edge_image = get_edges(blur_image, param['canny_lo'], param['canny_hi'])
  
  x, y = image.shape[1], image.shape[0]
  vertices = get_vertices(x, y, param['ax_coef'], param['bx_coef'],
                                param['cy_coef'], param['dy_coef'],
                                param['maxx_coef'], param['maxy_coef'],
                                param['startx_coef'])
  roi = get_roi(edge_image, vertices)  
  mask_image = mask(edge_image, roi)
  
  lines = get_lines(mask_image, param['rho'], param['theta_coef'],
                           param['min_votes'], param['min_line_length'],
                           param['max_line_gap'])
   
  #line_image = draw(lines, raw_image)  
  #result = overlap(line_image, raw_image)
 
  line_image2 = extrapolate_lines(lines, raw_image) 
  result = overlap(line_image2, raw_image)
  return result

In [16]:
white_output = 'white.mp4'
clip1 = VideoFileClip("/home/guruji/Downloads/advanced-lane-detection-master/A-BASIC-LANE-DETECTION/videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Moviepy - Building video white.mp4.
MoviePy - Writing audio in whiteTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video white.mp4



TypeError: must be real number, not NoneType

In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [19]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

OSError: MoviePy error: the file solidYellowLeft.mp4 could not be found!
Please check that you entered the correct path.

In [135]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [136]:
p = {
   #blur parameters
  'kernel_size': 3, 
    
   #canny transform parameters
  'canny_lo': 50, 
  'canny_hi': 150, 

  #region of interest parameters
  'ax_coef': 0.41,
  'bx_coef': 0.60,
  'cy_coef': 0.65,
  'dy_coef': 0.65,
  'maxy_coef': 0.9,
  'maxx_coef': 0.85,
  'startx_coef': 0.15,

  #hough parameters
  'rho': 1, 
  'theta_coef': 1, 
  'min_votes': 20, 
  'min_line_length': 50, 
  'max_line_gap': 20
}

def process(image):

  raw_image = np.copy(image)

  gray_image = gray(image)    
  blur_image = reduce_noise(gray_image, p['kernel_size'])
    
  edge_image = get_edges(blur_image, p['canny_lo'], p['canny_hi'])
  
  #return convert_to_color(edge_image) #check if canny parameters detect edges of lanes

  x = image.shape[1]
  y = image.shape[0]
  vertices = get_vertices(x, y, p['ax_coef'], p['bx_coef'],
                                p['cy_coef'], p['dy_coef'],
                                p['maxy_coef'], p['maxx_coef'], p['startx_coef'])
  
  roi = get_roi(edge_image, vertices)
  mask_image = mask(edge_image, roi)
  
  #return overlap(convert_to_color(roi), raw_image) #check if roi is good
    
  lines = get_lines(mask_image, p['rho'], p['theta_coef'],
                                p['min_votes'], p['min_line_length'],
                                p['max_line_gap'], )
   
  #line_image = draw(lines, raw_image, thresh = 0.5)  
  #result = overlap(line_image, raw_image)
 
  line_image2 = extrapolate_lines(lines, raw_image, 
                                  positive_thresh = 0.5, negative_thresh = -0.5) 
  result = overlap(line_image2, raw_image)
  
  return result

In [137]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:16<00:00, 15.36it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 6.35 s, sys: 1.68 s, total: 8.03 s
Wall time: 18 s


In [138]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))